# YOLO Training and Testing

This notebook shows how to train and test YOLO.

In [1]:
# Ensure Albumntations is not installed so it does not apply augmentations automatically.
#!pip uninstall albumentations -y

Found existing installation: albumentations 2.0.2
Uninstalling albumentations-2.0.2:
  Successfully uninstalled albumentations-2.0.2


In [1]:
import os
from ultralytics import YOLO
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import json
import ultralytics

ultralytics.checks()

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
Setup complete ✅ (32 CPUs, 62.5 GB RAM, 774.6/899.9 GB disk)


## Testing

The following show commands to perform testing on a trained YOLO model. Note:

- We set retina_masks=True to give better mask quality
- Confidence threshold is set to 0.5
- Set `model` to be the path to the best weights for the trained model
- Set `data` to be the path to the YOLO-formatted data to use for testing.

Note that in validation mode, by default YOLO will look for a directory under `data` called `val`. This is not what we want for **testing** - we want to run against the test set. Therefore, for testing, rename the `images/val` and `labels/val` directories to something else, and rename `images/test` to `images/val` and `labels/val` to `labels/test`. Then it will run validation using the test set.

YOLO's evaluation results differ from those of the official COCO API. To enable comparison with the other modesl, which use COCO evaluation, we also need to evaluate YOLO with COCO. The following function enables this.

In [2]:
# Adapted with changes from YOLO_RadioGalaxyNET/coco_evaluate.py in https://github.com/Nikhel1/Gal-YOLOv8
# Original copyright:
# Copyright (c) 2023 Nikhel
# MIT License text included in the README

def coco_evaluate(test_annos, predictions_path, per_cat=False):
    """
    Evaluate a trained YOLO format with the COCO evaluator.

    test_annos: The annotations file containing ground truth values
    predictions_path: Path to the location of the predictions from a trained YOLO model.
    per_cat: Whether to evaluate per category. Default is False.
    """

    # Ground truth annotations from the cleaned test set.
    coco_gt = COCO(str(test_annos))

    filename2id = {img['file_name']: img['id'] for img in coco_gt.dataset['images']}

    coco_preds = json.load(open(os.path.join(predictions_path, "predictions.json")))

    # Update image_id and category_id in coco_preds and save to predictions_new.json
    for idx in range(len(coco_preds)):
        coco_preds[idx]['image_id'] = filename2id[coco_preds[idx]['image_id'] + '.png']
        # coco_preds[idx]['category_id'] = coco_preds[idx]['category_id'] + 1

    with open(os.path.join(predictions_path, "predictions_new.json"), "w") as fp:
        json.dump(coco_preds, fp)

    coco_preds_new = coco_gt.loadRes(str(os.path.join(predictions_path, "predictions_new.json")))

    cats = coco_gt.loadCats(coco_gt.getCatIds())
    names = [cat['name'] for cat in cats]

    # Evaluate for bbox and segmentation
    # We use the sizes small, medium, large defined in Gupta et al, 2024
    for task in ['bbox', 'segm']:
        coco_eval = COCOeval(coco_gt, coco_preds_new, task)
        coco_eval.params.imgIds = coco_gt.getImgIds()
        coco_eval.params.areaRng = [[0 ** 2, 1e5 ** 2], [0 ** 2, 24 ** 2], [24 ** 2, 48 ** 2], [48 ** 2, 1e5 ** 2]]
        coco_eval.params.areaRngLbl = ['all', 'small', 'medium', 'large']
        coco_eval.evaluate()
        coco_eval.accumulate()
        coco_eval.summarize()

    if per_cat is True:
        for idx, name in enumerate(names):
            for task in ['bbox', 'segm']:
                print(f"Results for {name}")
                coco_eval = COCOeval(coco_gt, coco_preds_new, task)
                coco_eval.params.imgIds = coco_gt.getImgIds()
                coco_eval.params.catIds = idx + 1
                coco_eval.params.areaRng = [[0 ** 2, 1e5 ** 2], [0 ** 2, 24 ** 2], [24 ** 2, 48 ** 2],
                                            [48 ** 2, 1e5 ** 2]]
                coco_eval.params.areaRngLbl = ['all', 'small', 'medium', 'large']
                coco_eval.evaluate()
                coco_eval.accumulate()
                coco_eval.summarize()


""""
References
N. Gupta, Z. Hayder, R. P. Norris, M. Huynh, and L. Petersson, “Ra-
dioGalaxyNET: Dataset and novel computer vision algorithms for the
detection of extended radio galaxies and infrared hosts,” Publications of
the Astronomical Society of Australia, vol. 41, p. e001, 2024, publisher:
Cambridge University Press.
"""

'"\nReferences\nN. Gupta, Z. Hayder, R. P. Norris, M. Huynh, and L. Petersson, “Ra-\ndioGalaxyNET: Dataset and novel computer vision algorithms for the\ndetection of extended radio galaxies and infrared hosts,” Publications of\nthe Astronomical Society of Australia, vol. 41, p. e001, 2024, publisher:\nCambridge University Press.\n'

## Training

The following is a typical command to start training YOLO. Replace the path to `data` with the path to the data you want to use. Note the dataset has to be prepared in YOLO format. See the `README.md` in `data_processing` for instructions about that.

## Cleaned Dataset - YOLO v11 Large  Model

In [ ]:
%%bash
(yolo task=segment mode=train model=yolo11l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/YOLO_cleaned/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 mosaic=0 \
fliplr=0 patience=25)

Results saved to runs/segment/train13

                   all        410        605      0.691      0.757      0.752      0.727      0.692      0.758      0.756      0.623
                 FR-II        221        267      0.733      0.816      0.809      0.775       0.74      0.824      0.823      0.688
                  FR-I         78         83      0.748      0.644      0.742      0.725      0.748      0.644      0.746      0.626
                  FR-X         82         84      0.559       0.75      0.669      0.643      0.559       0.75      0.669      0.532
                     R        154        171      0.725      0.819      0.789      0.764      0.719      0.813      0.786      0.645

In [5]:
!yolo task=segment mode=val model=runs/segment/train13/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLO11l-seg summary (fused): 491 layers, 27,587,676 parameters, 0 gradients, 141.9 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val... 410 i
val: New cache created: /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.719      0.728      0.732      0.715      0.721       0.73      0.734      0.599
                 FR-II        221        267      0.754      0.801      0.791      0.767      0.761      0.809      0.802      0.647
                  FR-I         78         83      0.781      0.602      0.713      0.702      0.781      0.602      0.713      0.609
                  FR-X         82         84      0.598      0.726      0.658      0.642      0.598      0.726      0.658       0.53
                     R        15

In [7]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val2", per_cat=True)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.614
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.634
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.626
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.542
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.635
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.559
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.636
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.713
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### Cleaned Dataset - YOLO v8 Large Model

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/YOLO_cleaned/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 mosaic=0 \
fliplr=0 patience=25)

Results saved to runs/segment/train14

                   all        410        605      0.754      0.736      0.773      0.749      0.757       0.74      0.776      0.642
                 FR-II        221        267      0.787      0.831      0.839      0.804      0.801      0.846      0.852      0.707
                  FR-I         78         83      0.823      0.614      0.731      0.723      0.823      0.614      0.731      0.611
                  FR-X         82         84      0.659       0.69      0.707      0.681      0.659       0.69      0.707       0.57
                     R        154        171      0.746      0.807      0.813      0.787      0.746      0.807      0.813      0.679

#### Validation Set

In [10]:
!yolo task=segment mode=val model=runs/segment/train14/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.787      0.702      0.759      0.739      0.791      0.706      0.763      0.624
                 FR-II        221        267      0.812      0.809      0.814      0.785      0.827      0.824      0.827      0.668
                  FR-I         78         83       0.88      0.554       0.72      0.711       0.88      0.554       0.72      0.614
                  FR-X         82         84      0.693      0.667       0.71      0.688      0.693      0.667       0.71      0.575
                     R        154        171      0.764      0.778      0.794      0.773      0.764      0.778      0.79

In [11]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val4")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.618
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.640
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.634
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.556
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.654
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.394
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.618
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.687
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

#### Test Set

In [12]:
!yolo task=segment mode=val model=runs/segment/train14/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/labels/val.cache... 41
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        411        593      0.723      0.675      0.721       0.71      0.725      0.678      0.723      0.596
                 FR-II        225        277      0.808      0.838      0.861      0.843      0.819      0.848      0.868       0.71
                  FR-I         83         90      0.771        0.6      0.703      0.701      0.771        0.6      0.703      0.597
                  FR-X         81         86      0.671      0.593      0.645      0.637      0.671      0.593      0.645      0.535
                     R        124        140      0.639      0.671      0.674      0.658      0.639      0.671      0.67

In [13]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/test.json", "runs/segment/val15")

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.593
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.606
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.598
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.452
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.637
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.564
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.582
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.664
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets


## Testing Online Augmentations

### Defaults

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" \
epochs=150 batch=32 \
patience=25)

Results saved to runs/segment/train15

                   all        410        605       0.63       0.65      0.664      0.642      0.631      0.652      0.669      0.549
                 FR-II        221        267      0.666      0.742      0.735      0.703      0.673      0.749      0.756      0.614
                  FR-I         78         83      0.646      0.614      0.656      0.641      0.646      0.614      0.656      0.553
                  FR-X         82         84      0.574        0.5      0.547      0.536      0.574        0.5      0.547       0.45
                     R        154        171      0.632      0.743      0.719      0.687      0.632      0.743       0.72      0.579

In [14]:
!yolo task=segment mode=val model=runs/segment/train15/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.649      0.629      0.655      0.642      0.651       0.63      0.658      0.544
                 FR-II        221        267      0.682      0.723      0.716      0.698      0.689       0.73      0.728      0.594
                  FR-I         78         83      0.676      0.578      0.655       0.65      0.676      0.578      0.655      0.563
                  FR-X         82         84      0.597      0.476      0.531      0.527      0.597      0.476      0.531      0.441
                     R        154        171      0.643      0.737      0.717      0.695      0.643      0.737      0.71

In [16]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val6")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.520
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.534
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.532
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.486
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.548
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.288
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.529
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.617
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### Flips

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/YOLO_cleaned/datasets/data.yaml" \
epochs=150 batch=32 \
fliplr=1 flipud=1 patience=25)

                    all        410        605      0.578      0.513      0.543      0.196      0.509      0.437      0.432      0.104
                 FR-II        221        267      0.612      0.689      0.687      0.306      0.513      0.577      0.527      0.135
                  FR-I         78         83      0.639      0.422      0.513      0.194       0.64      0.407      0.479       0.11
                  FR-X         82         84      0.511      0.345      0.416       0.14      0.446      0.298      0.317     0.0861
                     R        154        171       0.55      0.596      0.557      0.143      0.438      0.468      0.404     0.0837
Speed: 0.1ms preprocess, 4.5ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/segment/train16

In [17]:
!yolo task=segment mode=val model=runs/segment/train16/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605       0.58      0.513      0.532      0.207      0.539      0.461      0.482      0.144
                 FR-II        221        267      0.613      0.693      0.667      0.323      0.549       0.61      0.576      0.176
                  FR-I         78         83      0.646      0.422      0.507      0.211      0.634      0.397      0.492      0.148
                  FR-X         82         84      0.514      0.345      0.408      0.145      0.477       0.31      0.364      0.128
                     R        154        171      0.546      0.591      0.545      0.148      0.498      0.526      0.49

In [18]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val7")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.153
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.419
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.040
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.082
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.180
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.139
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.200
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.220
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### Mosaic

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/YOLO_cleaned/datasets/data.yaml" \
epochs=150 batch=32 mosaic=0 \
patience=25)


                   all        410        605      0.748      0.689      0.733      0.705       0.75      0.691      0.736      0.605
                 FR-II        221        267      0.811      0.806      0.823      0.785      0.819      0.813      0.835      0.697
                  FR-I         78         83      0.742      0.578      0.652      0.638      0.742      0.578      0.652      0.537
                  FR-X         82         84      0.674      0.616      0.664      0.634      0.674      0.616      0.664      0.535
                     R        154        171      0.765      0.754      0.792      0.762      0.765      0.754      0.792      0.652

Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/segment/train17

In [19]:
!yolo task=segment mode=val model=runs/segment/train17/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.748      0.689      0.724      0.704       0.75       0.69      0.727      0.588
                 FR-II        221        267      0.811      0.806      0.804      0.777      0.819      0.813      0.814      0.658
                  FR-I         78         83      0.742      0.578      0.652       0.64      0.742      0.578      0.652      0.549
                  FR-X         82         84      0.674      0.616      0.658      0.639      0.674      0.616      0.658      0.528
                     R        154        171      0.765      0.754      0.783      0.761      0.765      0.754      0.78

In [20]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val8")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.599
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.622
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.615
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.595
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.603
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.454
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.621
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.691
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### HSV

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/YOLO_cleaned/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 patience=25)

                 all        410        605       0.71      0.682      0.728      0.695      0.712      0.685      0.732      0.593
                 FR-II        221        267      0.721      0.794      0.815      0.779      0.731      0.805      0.831      0.679
                  FR-I         78         83       0.68      0.614      0.686      0.665       0.68      0.614      0.686      0.569
                  FR-X         82         84      0.697      0.547      0.621       0.59      0.697      0.547      0.621      0.502
                     R        154        171      0.742      0.774      0.789      0.748      0.742      0.774      0.789      0.624
Speed: 0.1ms preprocess, 3.5ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/segment/train18

In [21]:
!yolo task=segment mode=val model=runs/segment/train18/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.702      0.689      0.713      0.692      0.705      0.692      0.716      0.583
                 FR-II        221        267      0.717      0.805      0.787      0.765      0.727      0.816        0.8      0.648
                  FR-I         78         83      0.662      0.614      0.678      0.661      0.662      0.614      0.678      0.573
                  FR-X         82         84      0.691       0.56      0.616      0.594      0.691       0.56      0.616      0.498
                     R        154        171      0.739      0.778      0.772      0.746      0.739      0.778      0.77

In [22]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val9")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.10s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.579
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.603
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.599
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.567
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.591
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.489
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.670
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

## Random Rotations and Combined with Cleaned Dataset

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/yolo_rotated/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 mosaic=0 \
fliplr=0 patience=25)

                   all        410        605      0.697       0.68      0.735      0.697      0.701      0.684      0.741      0.609
                 FR-II        221        267      0.716      0.772      0.802      0.758      0.729      0.787      0.824      0.675
                  FR-I         78         83      0.721       0.59      0.682      0.655      0.721       0.59      0.683      0.576
                  FR-X         82         84      0.635      0.619      0.666      0.631      0.636      0.619      0.666      0.546
                     R        154        171      0.717      0.739      0.789      0.745      0.716      0.738       0.79      0.641
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/segment/train19


#### Validation Set

In [23]:
!yolo task=segment mode=val model=runs/segment/train19/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.698       0.68      0.714      0.691      0.702      0.683      0.718      0.587
                 FR-II        221        267      0.715      0.772      0.775      0.743      0.729      0.787      0.791      0.635
                  FR-I         78         83      0.721       0.59      0.683      0.669      0.722       0.59      0.683      0.577
                  FR-X         82         84      0.635      0.619      0.636      0.619      0.636      0.619      0.636      0.527
                     R        154        171       0.72      0.738       0.76      0.732       0.72      0.737       0.7

In [24]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val10")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.587
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.612
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.606
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.536
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.614
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.370
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.604
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.674
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

#### Test Set

In [10]:
!yolo task=segment mode=val model=runs/segment/train19/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/labels/val.cache... 41
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        411        593      0.732      0.645      0.695      0.672      0.739      0.647      0.698      0.576
                 FR-II        225        277      0.797       0.78      0.824        0.8      0.814      0.794      0.836      0.686
                  FR-I         83         90      0.781      0.554      0.665      0.648       0.78      0.552      0.665      0.569
                  FR-X         81         86      0.624      0.558      0.566      0.541      0.627      0.558      0.566      0.466
                     R        124        140      0.728      0.686      0.723        0.7      0.734      0.686      0.72

In [11]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/test.json", "runs/segment/val14")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.576
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.601
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.594
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.409
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.618
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.564
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.678
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

## Random Rotations x3, with backgrounds and balancing, and Combined with Cleaned Dataset

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/yolo_rotated_x3/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 mosaic=0 \
fliplr=0 patience=25)

Results saved to runs/segment/train27

                   all        410        605        0.8      0.686      0.778      0.755      0.803      0.696      0.786      0.655
                 FR-II        221        267      0.804       0.79      0.833      0.807       0.82      0.809      0.862      0.721
                  FR-I         78         83       0.83      0.602       0.73      0.717      0.829      0.602      0.735      0.623
                  FR-X         82         84      0.766      0.607      0.739      0.713      0.761      0.619      0.739      0.609
                     R        154        171        0.8      0.743      0.808      0.781      0.801      0.755      0.808      0.666


#### Validation Set

In [4]:
!yolo task=segment mode=val model=runs/segment/train27/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.802      0.688      0.762      0.746      0.802      0.696      0.767      0.625
                 FR-II        221        267      0.803       0.79      0.817      0.798      0.819      0.809      0.839      0.679
                  FR-I         78         83       0.83      0.602      0.722      0.715      0.829      0.602      0.722      0.608
                  FR-X         82         84      0.775      0.616      0.718      0.702      0.761      0.619      0.718      0.584
                     R        154        171        0.8      0.743       0.79      0.771      0.801      0.754       0.7

In [5]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val20")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.641
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.660
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.655
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.564
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.678
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.420
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.631
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.706
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

#### Test Set

In [6]:
!yolo task=segment mode=val model=runs/segment/train27/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/labels/val.cache... 41
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        411        593      0.749      0.744      0.758      0.742      0.753      0.748      0.761      0.623
                 FR-II        225        277      0.797       0.85      0.841      0.826       0.81      0.864       0.85      0.694
                  FR-I         83         90      0.798      0.722      0.809      0.788      0.799      0.722      0.809      0.674
                  FR-X         81         86      0.691      0.663      0.641      0.623      0.692      0.663      0.641      0.527
                     R        124        140       0.71      0.743      0.742      0.731       0.71      0.743      0.74

In [9]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/test.json", "runs/segment/val21", per_cat=True)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.652
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.671
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.662
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.561
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.671
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.658
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.655
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.743
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

## Rotated by Multiples of 90, not combined

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/yolo_rotate_random_90/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 mosaic=0 \
fliplr=0 patience=25)

                   all        410        605      0.493      0.594      0.553      0.409       0.49      0.596      0.554      0.327
                 FR-II        221        267      0.529      0.809      0.698      0.524      0.529      0.809      0.707      0.415
                  FR-I         78         83      0.444       0.51      0.481      0.367      0.445      0.512      0.481      0.316
                  FR-X         82         84       0.38      0.595      0.442      0.321      0.372      0.595      0.433      0.244
                     R        154        171      0.618      0.463      0.591      0.424      0.614      0.468      0.592      0.333

Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/segment/train20

In [25]:
!yolo task=segment mode=val model=runs/segment/train20/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.762      0.234        0.5      0.392      0.766      0.237      0.503      0.342
                 FR-II        221        267      0.718      0.554      0.629      0.502      0.733      0.566      0.641      0.408
                  FR-I         78         83      0.737      0.169      0.456      0.365      0.737      0.169      0.456      0.331
                  FR-X         82         84      0.692      0.107      0.412      0.329      0.692      0.107      0.412      0.279
                     R        154        171        0.9      0.105      0.504      0.374        0.9      0.105      0.50

In [26]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val11")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.208
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.198
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.061
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.179
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.185
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.187
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### +2 FDA Transformed Galaxies

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/yolo_2_fda_galaxies/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 mosaic=0 \
fliplr=0 patience=25)

Results saved to runs/segment/train22

                   all        410        605      0.748      0.714      0.768      0.728      0.753      0.718      0.774      0.635
                 FR-II        221        267      0.784      0.809      0.839      0.794      0.802      0.828      0.862      0.715
                  FR-I         78         83      0.806      0.602      0.721      0.702      0.806      0.602      0.722      0.607
                  FR-X         82         84      0.666      0.631      0.705      0.666      0.666      0.631      0.705       0.57
                     R        154        171      0.735      0.813      0.808       0.75      0.735      0.813      0.808       0.65

### Validation Set

In [4]:
!yolo task=segment mode=val model=runs/segment/train22/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.747      0.714      0.751      0.724      0.751      0.719      0.756      0.609
                 FR-II        221        267      0.783      0.809       0.82      0.791      0.801      0.828      0.837      0.668
                  FR-I         78         83      0.807      0.603      0.698      0.687      0.807      0.603      0.698      0.591
                  FR-X         82         84      0.665      0.631       0.69       0.66      0.665      0.631       0.69      0.549
                     R        154        171      0.732      0.813      0.798      0.757      0.732      0.813      0.79

In [5]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val12")

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.617
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.649
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.638
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.537
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.656
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.467
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.625
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.700
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### Test Set

In [6]:
!yolo task=segment mode=val model=runs/segment/train22/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/labels/val... 411 imag
val: New cache created: /mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        411        593      0.752      0.687      0.711      0.686      0.754       0.69      0.713      0.572
                 FR-II        225        277      0.794      0.823      0.806      0.781      0.804      0.834      0.813      0.649
                  FR-I         83         90      0.842        0.6      0.727      0.706      0.842        0.6      0.727      0.599
                  FR-X         81         86      0.681      0.581      0.593      0.572      0.681      0.581      0.593       0.48
                     R        124  

In [10]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/test.json", "runs/segment/val13", per_cat=True)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.585
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.614
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.608
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.476
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.616
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.646
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.608
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.685
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### +2 Untransformed Galaxies (Baseline, Dataset Balancing)

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/yolo_2_untransformed_galaxies/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 mosaic=0 \
fliplr=0 patience=25)

Results saved to runs/segment/train24

                   all        410        605      0.753      0.698      0.758      0.739      0.757      0.703      0.766      0.643
                 FR-II        221        267      0.767      0.816      0.825      0.802      0.785      0.835      0.857      0.725
                  FR-I         78         83      0.731      0.602      0.724      0.709      0.731      0.602      0.724      0.622
                  FR-X         82         84      0.679      0.619      0.655      0.634      0.679      0.619      0.655      0.536
                     R        154        171      0.834      0.754      0.829       0.81      0.834      0.754      0.829      0.691

### Validation Set

In [11]:
!yolo task=segment mode=val model=runs/segment/train24/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.752      0.698       0.74      0.726      0.757      0.703      0.747      0.606
                 FR-II        221        267      0.765      0.818      0.807      0.788      0.783      0.836      0.835      0.669
                  FR-I         78         83       0.73      0.602      0.692      0.683       0.73      0.602      0.692      0.586
                  FR-X         82         84      0.679      0.619       0.65      0.638      0.679      0.619       0.65      0.524
                     R        154        171      0.834      0.754       0.81      0.795      0.834      0.754       0.8

In [12]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val18")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.615
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.632
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.624
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.560
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.618
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.536
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.613
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.691
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### Test Set

In [13]:
!yolo task=segment mode=val model=runs/segment/train24/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/labels/val.cache... 41
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        411        593      0.745       0.69      0.708      0.698      0.749      0.695      0.711      0.579
                 FR-II        225        277      0.764       0.83        0.8      0.788      0.781      0.848      0.813      0.654
                  FR-I         83         90      0.779      0.678      0.719      0.706      0.779      0.678      0.719      0.593
                  FR-X         81         86      0.667      0.547      0.596       0.59      0.667      0.547      0.596      0.495
                     R        124        140       0.77      0.707      0.717       0.71       0.77      0.707      0.71

In [11]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/test.json", "runs/segment/val19", per_cat=True)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.588
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.600
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.598
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.501
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.617
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.647
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.604
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.689
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### +2 VAE Galaxies

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/yolo_2_vae_galaxies/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 mosaic=0 \
fliplr=0 patience=25)

Results saved to runs/segment/train30

                   all        410        605      0.725      0.681       0.74      0.718      0.728      0.684      0.744      0.618
                 FR-II        221        267      0.729      0.783      0.806      0.782      0.743      0.798      0.824      0.695
                  FR-I         78         83       0.75      0.602      0.723      0.708       0.75      0.602      0.723      0.605
                  FR-X         82         84       0.66      0.577      0.646      0.623       0.66      0.577      0.646       0.53
                     R        154        171       0.76       0.76      0.783      0.758       0.76       0.76      0.783      0.643


### Validation Set

In [4]:
!yolo task=segment mode=val model=runs/segment/train30/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.739      0.665      0.716        0.7      0.743      0.669       0.72      0.587
                 FR-II        221        267      0.746      0.779      0.789       0.77       0.76      0.794      0.804      0.652
                  FR-I         78         83      0.766       0.59      0.727      0.722      0.766       0.59      0.727      0.619
                  FR-X         82         84      0.676      0.548      0.609      0.589      0.676      0.548      0.609      0.491
                     R        154        171       0.77      0.743      0.741       0.72       0.77      0.743      0.74

In [5]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val23")

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.575
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.593
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.587
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.460
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.616
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.416
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.579
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.651
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### Test Set

In [6]:
!yolo task=segment mode=val model=runs/segment/train30/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/labels/val.cache... 41
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        411        593      0.724       0.69      0.727      0.716      0.724      0.691      0.728      0.595
                 FR-II        225        277      0.787      0.851      0.836      0.815       0.79      0.855      0.839      0.681
                  FR-I         83         90      0.818      0.656      0.747      0.745      0.818      0.656      0.747      0.627
                  FR-X         81         86      0.598      0.547      0.589       0.58      0.598      0.547      0.589      0.484
                     R        124        140      0.693      0.707      0.736      0.725      0.693      0.707      0.73

In [7]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/test.json", "runs/segment/val24")

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.606
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.619
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.612
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.459
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.645
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.533
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.605
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.683
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### +2 GAN Galaxies

In [ ]:
%%bash
(yolo task=segment mode=train model=yolov8l-seg.pt \
data="/mnt/data/rgn_ijcnn/yolo/yolo_2_gan_galaxies/datasets/data.yaml" \
epochs=150 batch=32 \
hsv_h=0 hsv_s=0 hsv_v=0 mosaic=0 \
fliplr=0 patience=25)


Results saved to runs/segment/train25

                   all        410        605      0.723      0.701      0.743      0.721      0.738       0.69      0.748      0.624
                 FR-II        221        267      0.755      0.828      0.799       0.77      0.771      0.831      0.821      0.685
                  FR-I         78         83      0.756      0.671      0.741      0.722      0.779      0.651      0.741      0.637
                  FR-X         82         84      0.616      0.512      0.625      0.606      0.621      0.507      0.625      0.511
                     R        154        171      0.766      0.795      0.808      0.788       0.78      0.772      0.804      0.665

### Validation Set

In [6]:
!yolo task=segment mode=val model=runs/segment/train25/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_cleaned/datasets/labels/val.cache...
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        410        605      0.758      0.664      0.727       0.71      0.762      0.668      0.732      0.592
                 FR-II        221        267      0.788      0.805      0.777      0.754      0.802       0.82      0.797      0.639
                  FR-I         78         83      0.783      0.614      0.737      0.727      0.783      0.614      0.737      0.622
                  FR-X         82         84      0.658      0.476      0.597      0.581      0.658      0.476      0.597      0.475
                     R        154        171      0.803      0.761      0.797      0.779      0.803      0.761      0.79

In [8]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/val.json", "runs/segment/val16")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.10s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.592
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.609
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.605
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.561
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.596
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.543
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.602
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.665
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

### Test Set

In [9]:
!yolo task=segment mode=val model=runs/segment/train25/weights/best.pt retina_masks=True conf=0.5 data="/mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/data.yaml" save_json=True

Ultralytics 8.3.68 🚀 Python-3.12.8 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4090, 24209MiB)
YOLOv8l-seg summary (fused): 295 layers, 45,914,972 parameters, 0 gradients, 220.1 GFLOPs
val: Scanning /mnt/data/rgn_ijcnn/yolo/yolo_test/datasets/labels/val.cache... 41
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        411        593      0.707      0.675      0.717      0.709      0.709      0.678      0.718      0.593
                 FR-II        225        277      0.768       0.87      0.849      0.836      0.777      0.881      0.856      0.696
                  FR-I         83         90      0.713      0.633      0.728      0.725      0.713      0.633      0.728      0.621
                  FR-X         81         86      0.641      0.477      0.579      0.574      0.641      0.477      0.579      0.485
                     R        124        140      0.706      0.721       0.71      0.703      0.706      0.721       0.7

In [10]:
coco_evaluate("/mnt/data/rgn_ijcnn/cleaned/annotations/test.json", "runs/segment/val17")

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.596
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.606
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.601
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.447
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.630
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.663
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.592
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets